# Tarea Apache Spark
>Importa las librerías necesarias dónde sea necesario


In [1]:
import pandas as pd 
from pyspark.sql import SparkSession


### SparkSession
>Crea un SparkSession para comenzar la tarea

In [2]:
#Crea un objeto de tipo SparkSession
spark = SparkSession.builder.appName("Practica Spark").getOrCreate()

23/12/09 21:15:53 WARN Utils: Your hostname, Gerardos-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
23/12/09 21:15:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/09 21:15:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Crear un DataFrame
>Lee el csv datosTarea.csv, mételo a un DF y muéstralo.

In [3]:
data = spark.read.csv('datosTarea.csv',header=True,inferSchema=True)
data.show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|  402269|         33| 12125|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|  569480|         49| 12001|
|    3|0bFED1ADAE4bcC1|          Hester Ltd|http://sullivan-r...|          

### Filtro de datos
>Consigue todas las empresas que empiecen con 'M' y tengan entre 4000 y 7000 empleados. Sólo muestra los nombres y el número de empleados.

In [4]:
data.filter((data['Name'].startswith('M')) & (data['Number of employees'] >= 4000) & (data['Number of employees'] <= 7000)).select('Name','Number of employees').show()

+--------------------+-------------------+
|                Name|Number of employees|
+--------------------+-------------------+
|Mckinney, Riley a...|               4952|
|       Mcintosh-Mora|               4389|
|     Mckenzie-Melton|               4589|
|          Massey LLC|               5004|
|        Mays-Preston|               5786|
+--------------------+-------------------+



>Consigue todos los países que no inicien con las letras 'b', 's' y 'm', pero que tampoco tengan un netword mayor a 500000. Muestra el nombre de la compañía, el país y el networth.

In [5]:
data.filter((data['Country'].startswith('b') == False) & (data['Country'].startswith('s') == False) & (data['Country'].startswith('m') == False) & (data['Networth'] <= 500000)).select('Name','Country','Networth').show()

+--------------------+--------------------+--------+
|                Name|             Country|Networth|
+--------------------+--------------------+--------+
|         Ferrell LLC|    Papua New Guinea|  402269|
|      Holder-Sellers|        Turkmenistan|  105914|
|Keller, Campos an...|             Liberia|  329130|
|      Pacheco-Spears|              Sweden|   88435|
|         Harrell LLC|          Guadeloupe|  251274|
|         Jenkins Inc|        South Africa|  139725|
|Dickson, Richmond...|      Czech Republic|  359030|
|        Prince-David|    Christmas Island|  120289|
|         Rivas Group|           Australia|  477824|
|Sloan, Mays and W...|                Chad|   41975|
|Glass, Barrera an...|     Kyrgyz Republic|  300150|
|          Pineda-Cox|             Bolivia|  150880|
|Baker, Mccann and...|               Kenya|  188370|
|            Hahn PLC|             Belarus|  427224|
|Valentine, Fergus...|              Jersey|  412274|
|           Walls LLC|          Cape Verde|  1

### Funciones
Crea una función con @pandas_udf que que le reste a los profits la media en cada renglón. Crea una nueva columna que muestre los resultados.

In [6]:
from pyspark.sql.functions import pandas_udf

@pandas_udf('long')
def subtract_mean(series: pd.Series) -> pd.Series:
    avg = series.mean()
    return series - avg
data = data.withColumn("avg_substraction", subtract_mean(data['Networth']))

data.select('Name','avg_substraction').show()


+--------------------+----------------+
|                Name|avg_substraction|
+--------------------+----------------+
|         Ferrell LLC|         -286416|
|Mckinney, Riley a...|         -119205|
|          Hester Ltd|          -80680|
|      Holder-Sellers|         -582771|
|         Mayer Group|          216363|
|      Henry-Thompson|         -123575|
|      Hansen-Everett|          211993|
|       Mcintosh-Mora|         -183951|
|            Carr Inc|          250518|
|          Gaines Inc|          426584|
|          Kidd Group|          170708|
|        Crane-Clarke|          347578|
|Keller, Campos an...|         -359555|
|         Glover-Pope|          355398|
|      Pacheco-Spears|         -600250|
|         Hodge-Ayers|          289733|
|Bowers, Guerra an...|          114703|
|     Mckenzie-Melton|         -160950|
|         Branch-Mann|          226828|
|      Weiss and Sons|            -525|
+--------------------+----------------+
only showing top 20 rows



### Grouping data
>Agrupa por industry y muestra cuáles son las empresas con el profit más alto. Muestra los primeros tres.

In [7]:
# Agrupa por industry y muestra cuáles son las empresas con el profit más alto. Muestra los primeros tres.
data.groupBy('Industry').max('Profit').show(3)

+--------------------+-----------+
|            Industry|max(Profit)|
+--------------------+-----------+
|Primary / Seconda...|      15424|
|     Broadcast Media|      12510|
|           Wholesale|       9754|
+--------------------+-----------+
only showing top 3 rows



>Agrupa por industry y calcula el promedio de empleados que tienen

In [8]:
#Agrupa por industry y calcula el promedio de empleados que tienen
data.groupBy('Industry').avg('Number of employees').show()

+--------------------+------------------------+
|            Industry|avg(Number of employees)|
+--------------------+------------------------+
|Primary / Seconda...|       6457.666666666667|
|     Broadcast Media|                  2589.0|
|           Wholesale|                  5010.0|
|Investment Manage...|                  3133.5|
|    Food / Beverages|                  9011.0|
|  Gambling / Casinos|                  4873.0|
|Logistics / Procu...|                  4155.0|
|            Maritime|                   769.0|
|            Wireless|                  6146.0|
|Education Management|                   339.0|
|       Arts / Crafts|                  2800.0|
|           Insurance|                  1215.0|
|  Financial Services|                  5157.0|
|Business Supplies...|                  9097.0|
|Consumer Electronics|                  5022.0|
|       Public Safety|                  5287.0|
|Information Techn...|                  3934.0|
|Civic / Social Or...|                  

### SQL
>Usando Spark SQL, obtén cuántas empresas se fundaron despúes del 2000.

In [9]:
data.createOrReplaceTempView("Companies")

result = spark.sql("SELECT COUNT(*) as count FROM companies WHERE Founded > 2000")
result.show()


+-----+
|count|
+-----+
|   38|
+-----+



### ML Regresión Lineal
>Con número de empleados, networth y stock price, obtén una predicción del profit a través de una regresión lineal.

In [10]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline


assembler = VectorAssembler(
    inputCols=["Number of employees", "Networth", "stock_price"],
    outputCol="ind_features"
)


In [11]:

output=assembler.transform(data)
output.show()


+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+----------------+--------------------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|avg_substraction|        ind_features|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+----------------+--------------------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|  402269|         33| 12125|         -168689|[3498.0,402269.0,...|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...| 

In [12]:

finalized_data = output.select("ind_features","profit")


In [13]:
train_data,test_data=finalized_data.randomSplit([0.75, 0.25])


In [14]:

# Crear el modelo de regresión lineal
regressor = LinearRegression(featuresCol="ind_features", labelCol="profit")
regressor = regressor.fit(train_data)


23/12/09 21:16:05 WARN Instrumentation: [6ca98f33] regParam is zero, which might cause numerical instability and overfitting.
23/12/09 21:16:05 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/09 21:16:05 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/12/09 21:16:05 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [15]:
predictions = regressor.transform(test_data)

In [16]:
pred_results=regressor.evaluate(test_data)

>Una vez que obtengas los resultados, a través del api de pandas, conviértelo en un pandas on spark DataFrame y pásalo a csv.

In [17]:
pred_results.predictions.show()

+--------------------+------+------------------+
|        ind_features|profit|        prediction|
+--------------------+------+------------------+
|[236.0,27139.0,27.0]| 12510|11676.862921061422|
|[346.0,39790.0,21.0]| 11762|13260.570171669835|
|[563.0,64744.0,44.0]| 17766|11839.179813753355|
|[769.0,88435.0,34.0]| 12677| 13394.75354557667|
|[869.0,99934.0,15.0]|  7787|11612.528427298796|
|[2215.0,254724.0,...|  6746|11909.976425305938|
|[2862.0,329130.0,...| 17043|13465.911980653815|
|[3816.0,438839.0,...|  6840| 12019.95314790574|
|[3881.0,446314.0,...|  9861|12023.733710356684|
|[3934.0,452409.0,...| 13879|11748.650092511902|
|[4044.0,465059.0,...| 18466|12083.789876165136|
|[4389.0,504734.0,...|  5830|11850.977547531207|
|[4516.0,519339.0,...|  5029|11900.510583629779|
|[5038.0,579370.0,...| 11608|13482.892945364303|
|[5287.0,608005.0,...| 17121|  13590.0974847014|
|[6874.0,790509.0,...| 11139|12206.243353856153|
|[7664.0,881359.0,...|  6466|12252.191729121909|
|[7832.0,900679.0,..